In [3]:
import pandas as pd
import altair as alt
state_to_fips = {
    "Alabama": 1, "Alaska": 2, "Arizona": 4, "Arkansas": 5,
    "California": 6, "Colorado": 8, "Connecticut": 9, "Delaware": 10,
    "Florida": 12, "Georgia": 13, "Hawaii": 15, "Idaho": 16,
    "Illinois": 17, "Indiana": 18, "Iowa": 19, "Kansas": 20,
    "Kentucky": 21, "Louisiana": 22, "Maine": 23, "Maryland": 24,
    "Massachusetts": 25, "Michigan": 26, "Minnesota": 27, "Mississippi": 28,
    "Missouri": 29, "Montana": 30, "Nebraska": 31, "Nevada": 32,
    "New Hampshire": 33, "New Jersey": 34, "New Mexico": 35, "New York": 36,
    "North Carolina": 37, "North Dakota": 38, "Ohio": 39, "Oklahoma": 40,
    "Oregon": 41, "Pennsylvania": 42, "Rhode Island": 44, "South Carolina": 45,
    "South Dakota": 46, "Tennessee": 47, "Texas": 48, "Utah": 49,
    "Vermont": 50, "Virginia": 51, "Washington": 53, "West Virginia": 54,
    "Wisconsin": 55, "Wyoming": 56
}

url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
df = pd.read_csv(url)

state_counts = (
    df.groupby('state', as_index=False)
    .size()
    .rename(columns={'size': 'count'})
)
state_counts['state_id'] = state_counts['state'].map(state_to_fips)

state_counts.to_csv("bfro_reports_processed.csv", index=False)

In [5]:
state_to_code = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR",
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE",
    "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", "Idaho": "ID",
    "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS",
    "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV",
    "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY",
    "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK",
    "Oregon": "OR", "Pennsylvania": "PA", "Rhode Island": "RI", "South Carolina": "SC",
    "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT",
    "Vermont": "VT", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV",
    "Wisconsin": "WI", "Wyoming": "WY"
}

url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
df = pd.read_csv(url)

df["state_code"] = df["state"].map(state_to_code)

df.to_csv("bfro_reports_processed.csv", index=False)

In [9]:
# %% [markdown]
# # UFO Sightings Analysis
# Data source: https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv

# %%
import pandas as pd
import altair as alt
from datetime import datetime
alt.data_transformers.disable_max_rows()

# %%
# Load and prepare data
url = "https://raw.githubusercontent.com/UIUC-iSchool-DataViz/is445_data/main/bfro_reports_fall2022.csv"
ufo = pd.read_csv(url)

# Convert date to datetime and extract year
ufo['date'] = pd.to_datetime(ufo['date'], errors='coerce')
ufo = ufo.dropna(subset=['date']).copy()
ufo['year'] = ufo['date'].dt.year

# Create line chart data
state_counts = ufo.groupby(['state', 'year']).size().reset_index(name='count')

# Create choropleth data
state_totals = ufo.groupby('state').size().reset_index(name='total')  # Fixed typo here

# %%
# Line Chart
line_chart = alt.Chart(state_counts).mark_line().encode(
    x='year:O',
    y='count:Q',
    color=alt.value('steelblue')
).add_params(
    alt.selection_point(fields=['state'], bind=alt.binding_select(
        options=state_counts.state.unique().tolist(),
        name='Select State:'
    ))
).transform_filter(
    'datum.state == param_5.state'
).properties(
    width=800,
    height=400,
    title='UFO Sightings by State Over Time'
)

line_chart.save('line_chart.json')

# %%
# Choropleth Map
states = alt.topo_feature('https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json', 'states')

choropleth = alt.Chart(states).mark_geoshape().encode(
    color='total:Q',
    tooltip=['state:N', 'total:Q']
).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(state_totals, 'state', ['total'])
).project(
    type='albersUsa'
).properties(
    width=800,
    height=400,
    title='Total UFO Sightings by State'
)

choropleth.save('choropleth.json')